In [ ]:
#1.Constructing The Mixed Network Collection
import networkx as nx
import pickle
import os

def process_networks(start_year=1893, end_year=2020):
    base_path = "C:\\Network\\"
    yearly_path = os.path.join(base_path, "Network_Yearly")
    all_path = os.path.join(base_path, "Network_All")
    mixed_path = os.path.join(base_path, "Network_Mixed")

    if not os.path.exists(mixed_path):
        os.makedirs(mixed_path)

    for year in range(start_year, end_year + 1):
        
        yearly_file = os.path.join(yearly_path, f"network_yearly_{year}.pickle")
        all_file = os.path.join(all_path, f"network_{year}.pickle")
        mixed_file = os.path.join(mixed_path, f"network_mixed_{year}.pickle")    
       
        with open(yearly_file, 'rb') as f:         # Access the annual collaboration networks
            yearly_network = pickle.load(f)
        
        with open(all_file, 'rb') as f:           # Access the cumulative collaboration networks
            all_network = pickle.load(f)

        # Assign the degree values of authors in the cumulative collaboration network  
        # of the current year to the corresponding nodes in the annual collaboration network. 
        
        for node in yearly_network.nodes:      
            if node in all_network.nodes:
                yearly_network.nodes[node]['degree_all'] = all_network.degree[node]
            else:
                yearly_network.nodes[node]['degree_all'] = 0

        with open(mixed_file, 'wb') as f:
            pickle.dump(yearly_network, f)

process_networks()


In [2]:
#2. Calculating the Assortativity Coefficient of The Mixed Networks
import networkx as nx
import pickle
import os
import csv
import numpy as np


def clean_network(G, attribute):  # Remove nodes with NaN or infinite attribute values.
    nodes_to_remove = [n for n, v in nx.get_node_attributes(G, attribute).items() if np.isnan(v) or np.isinf(v)]
    G.remove_nodes_from(nodes_to_remove)

def safe_attribute_assortativity_coefficient(G, attribute):  # Calculate the assortativity coefficient, handling NaN and infinite values.
    clean_network(G, attribute)
    if len(G.nodes) == 0:  
        return np.nan
    return nx.attribute_assortativity_coefficient(G, attribute)

def calculate_assortativity(start_year=1893, end_year=2020):
    base_path = "C:\\Network\\"
    mixed_path = os.path.join(base_path, "Network_Mixed")
    analysis_path = os.path.join(base_path, "Network_Analysis")
    output_file = os.path.join(analysis_path, "year_mixed_assortativity.csv")

    results = []

    for year in range(start_year, end_year + 1):
        print(year)
        mixed_file = os.path.join(mixed_path, f"network_mixed_{year}.pickle")

        try:
            with open(mixed_file, 'rb') as mf:
                network = pickle.load(mf)
        except FileNotFoundError as e:
            print(f"File not found: {e}")
            continue

        # Calculate the assortativity of the degree_all attribute values of nodes in the mixed networks
        degree_all_assortativity = safe_attribute_assortativity_coefficient(network, 'degree_all')

        results.append([year, degree_all_assortativity])

    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['year', 'degree_all_assortativity'])
        writer.writerows(results)

if __name__ == "__main__":
    calculate_assortativity()
